Inspirations:

https://keras.io/examples/vision/autoencoder/

https://blog.keras.io/building-autoencoders-in-keras.html



---



We're using MNIST digits, and we're discarding the labels (since we're only interested in encoding/decoding the input images).



In [ ]:
from keras.datasets import mnist
import numpy as np
(x_train, _), (x_test, y_test) = mnist.load_data()

We will normalize all values between 0 and 1 and we will flatten the 28x28 images into vectors of size 784.

In [ ]:
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))
print(x_train.shape)
print(x_test.shape)



---



We'll start simple, with a single fully-connected neural layer as encoder and as decoder:

In [ ]:
import keras
from keras import layers
from keras import regularizers

# This is the size of our encoded representations
encoding_dim = 32  # 32 floats -> compression of factor 784/32=24.5, assuming the input is 784 floats
# This is our input image
input_img = keras.Input(shape=(784,))

In [ ]:
# "encoded" is the encoded representation of the input
encoded = layers.Dense(encoding_dim, activation='relu')(input_img)
# encoded = layers.Dense(encoding_dim, activation='relu', activity_regularizer=regularizers.l1(10e-5))(input_img)
# "decoded" is the lossy reconstruction of the input
decoded = layers.Dense(784, activation='sigmoid')(encoded)

Let's also create a separate autoencoder and encoder model:

In [ ]:
# This model maps an input to its reconstruction
autoencoder = keras.Model(input_img, decoded)

In [ ]:
# This model maps an input to its encoded representation
encoder = keras.Model(input_img, encoded)

As well as the decoder model:

In [ ]:
# This is our encoded (32-dimensional) input
encoded_input = keras.Input(shape=(encoding_dim,))
# Retrieve the last layer of the autoencoder model
decoder_layer = autoencoder.layers[-1]
# Create the decoder model
decoder = keras.Model(encoded_input, decoder_layer(encoded_input))

In [ ]:
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

In [ ]:
autoencoder.fit(x_train, x_train,
                epochs=50,
                batch_size=256,
                shuffle=True,
                validation_data=(x_test, x_test))

 We can visualize the reconstructed inputs and the encoded representations.

In [ ]:
# Encode and decode some digits
# Note that we take them from the *test* set
encoded_imgs = encoder.predict(x_test)
decoded_imgs = decoder.predict(encoded_imgs)

In [ ]:
# Use Matplotlib (don't ask)
import matplotlib.pyplot as plt

n = 10  # How many digits we will display
plt.figure(figsize=(20, 4))
for i in range(n):
    # Display original
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(x_test[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # Display reconstruction
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(decoded_imgs[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

The top row is the original digits, and the bottom row is the reconstructed digits.

In [ ]:
# the sparsity of the encoded representations. encoded_imgs.mean()
encoded_imgs.mean()



---



In [ ]:
from sklearn.decomposition import PCA
# Użyj PCA do wizualizacji klastrów
pca = PCA(n_components=2)  # Redukcja wymiarów za pomocą PCA do 2 komponentów
encoded_imgs_2d = pca.fit_transform(encoded_imgs)  # Przekształcenie zakodowanych wektorów do 2D

In [ ]:
from sklearn.cluster import KMeans
# Użyj KMeans do analizy klastrów
kmeans = KMeans(n_clusters=10, random_state=0)  # Algorytm KMeans z 10 klastrami
clusters = kmeans.fit_predict(encoded_imgs)     # Dopasowanie modelu KMeans i przypisanie klastrów

In [ ]:
# Wizualizacja klastrów
plt.figure(figsize=(10, 8))  # Ustawienie rozmiaru wykresu
plt.scatter(encoded_imgs_2d[:, 0], encoded_imgs_2d[:, 1], c=clusters, cmap='tab10')  # Wykres rozproszenia z kolorami odpowiadającymi klastrom
plt.colorbar()  # Dodanie paska kolorów
plt.show()  # Wyświetlenie wykresu

In [ ]:
# Mapowanie klastrów na oryginalne etykiety
def map_clusters_to_labels(clusters, labels):
    label_map = {}  # Słownik do mapowania klastrów na etykiety
    for cluster in np.unique(clusters):  # Iteracja po unikalnych klastrach
        indices = np.where(clusters == cluster)  # Znalezienie indeksów próbek należących do danego klastra
        true_labels = labels[indices]  # Pobranie prawdziwych etykiet dla tych próbek
        most_common_label = np.bincount(true_labels).argmax()  # Znalezienie najczęściej występującej etykiety
        label_map[cluster] = most_common_label  # Przypisanie najczęstszej etykiety do klastra
    print(f"Przypisanie najczęstszej etykiety do klastra {label_map}")
    return np.vectorize(label_map.get)(clusters)  # Mapowanie klastrów na etykiety

In [ ]:
from sklearn.metrics import accuracy_score
predicted_labels = map_clusters_to_labels(clusters, y_test)  # Mapowanie klastrów na przewidywane etykiety
accuracy = accuracy_score(y_test, predicted_labels)  # Obliczenie dokładności klastrowania
print(f"accuracy={accuracy}")

In [ ]:
# Wyświetlenie macierzy konfuzji
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, predicted_labels)  # Macierz konfuzji dla prawdziwych i przewidywanych etykiet
plt.figure(figsize=(10, 8))  # Ustawienie rozmiaru wykresu
import seaborn as sns
sns.heatmap(cm, annot=True, fmt="d", cmap='Blues')  # Wykres macierzy konfuzji
plt.xlabel('Predicted')  # Etykieta osi x
plt.ylabel('True')  # Etykieta osi y
plt.show()  # Wyświetlenie wykresu